In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.datasets as D
import torch.nn.functional as F
import numpy as np
import pickle
from matplotlib import pyplot as plt
from torchvision.io import read_image
from torchvision import transforms
from torch.utils.data import DataLoader
import fiona
import rasterio
import rasterio.mask
import pathlib
import pandas as pd
import os
import rioxarray
import sys
from tqdm import tqdm
sys.path.append("../src/")
from networks.ModisTempNet import ModisTempNet

In [7]:
root="./Ventanas"
classification = pd.read_csv(f'{root}/database.csv')
unique_classes = classification.clase.unique()
translation = {}
for i, j in zip(unique_classes, [k for k in range(len(unique_classes))]):
    translation[i] = j
n_classes = len(translation)

In [26]:
imgs = os.listdir(f"{root}/")
mins = {}
maxes = {}
for i in imgs:
    if i.split(".")[1] == "tif" and len(i.split(".")) == 2:
        src = rasterio.open(root + "/" + i)
        mask = src.dataset_mask()
        data = src.read()
        for band in range(data.shape[0]):
            if band in maxes.keys():

                if np.max(data[band]) > maxes[band]:
                    maxes[band] = np.max(data[band])
                if np.min(data[band]) < mins[band]:
                    mins[band] = np.min(data[band])
            else:
                maxes[band] = np.max(data[band])
                mins[band] = np.min(data[band])
for band in range(data.shape[0]):
    mins[band] -= 1

In [27]:
min(mins.values())

-4582.60107421875

In [25]:
maxes

{0: 359.02267,
 1: 72.81767,
 2: 42.0,
 3: 130.8,
 4: 4945.9272,
 5: 16000.903,
 6: 15969.746,
 7: 15712.811,
 8: 15545.911,
 9: 15580.33,
 10: 15392.989,
 11: 15483.522,
 12: 15272.251,
 13: 6378.4575,
 14: 5336.0,
 15: 99.80616,
 16: 0.9812146,
 17: 97.37583,
 18: 95.70563,
 19: 0.96271145,
 20: 3.4386322,
 21: 1.0,
 22: 1.0,
 23: 3.631598,
 24: 13.298929,
 25: 427.47595,
 26: 1.0,
 27: 1.0,
 28: 6.4465914,
 29: 9293.079,
 30: 9148.152,
 31: 8836.59,
 32: 7984.9365,
 33: 7817.0737,
 34: 7061.9585,
 35: 7278.897,
 36: 6360.9575,
 37: 6572.528,
 38: 6648.986,
 39: 93.71342,
 40: 1.0,
 41: 95.589714,
 42: 94.02116,
 43: 1.4169925,
 44: 5.499763,
 45: 1.0,
 46: 1.0,
 47: 4.10443,
 48: 12.767155,
 49: 267.63876,
 50: 0.9991404,
 51: 0.9773291,
 52: 6.4278984,
 53: 16.763245,
 54: 7.222836,
 55: 1001.43787,
 56: 5.2855086,
 57: 3.4812465,
 58: 1.0886546,
 59: 21.751287,
 60: 9.262102,
 61: 2047.4321,
 62: 4.7523537,
 63: 3.5345469,
 64: 0.8902169,
 65: 5.7772074,
 66: 0.9996719,
 67: 0.933